In [0]:
!pip install PyDrive #下載PyDrive ------google colab需要-------

In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#import連結Drive必要元件 ------google colab需要-------

In [0]:
download = drive.CreateFile({'id': '1i6LrlBCOxZWBwQhPVLVwQqI0a6A6_IDF'})

#連結drive,給他aoi.zip的共用連結id 

download.GetContentFile('aoi.zip') 

!unzip aoi.zip  #解壓縮
!unzip train_images.zip
!unzip test_images.zip

#------google colab需要-------

In [4]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD

Using TensorFlow backend.


In [11]:

train = pd.read_csv('train.csv')
#AOI圖檔都為灰色

#把圖片丟進去轉換

train_image = []

for i in tqdm(range(train.shape[0])):

    img = image.load_img('train_images/'+train['ID'][i], target_size=(100,100,1), grayscale=True) #圖檔名

    img = image.img_to_array(img)

    img = img/255

    train_image.append(img)

X = np.array(train_image)

  0%|          | 0/2528 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/utils.py:104: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
100%|██████████| 2528/2528 [00:07<00:00, 320.42it/s]


In [0]:
y=train['Label'].values
y = to_categorical(y) #one-hot 類別矩陣
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)


#圖片變形
gen = ImageDataGenerator(featurewise_center=False,
                         samplewise_center=False,
                         rotation_range=8, 
                         width_shift_range=0.08, 
                         shear_range=0.3,
                         height_shift_range=0.08, 
                         zoom_range=0.08,
                         data_format="channels_last")

gen.fit(X_train)
train_generator = gen.flow(X_train, y_train, batch_size=300)

test_gen = ImageDataGenerator(data_format="channels_last")
test_gen.fit(X_test)
test_generator = test_gen.flow(X_test, y_test, batch_size=300)

In [13]:
# 類似VGG的卷積神經網路下去修改
model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(100, 100, 1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(96, (3, 3), activation='relu'))
model.add(Conv2D(96, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

sgd = SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=50,validation_data=(X_test, y_test))




Train on 2022 samples, validate on 506 samples
Epoch 1/100
2022/2022 [==============================] - 5s 3ms/step - loss: 1.7143 - acc: 0.2423 - val_loss: 1.6919 - val_acc: 0.2806
Epoch 2/100
2022/2022 [==============================] - 3s 1ms/step - loss: 1.6647 - acc: 0.2681 - val_loss: 1.6432 - val_acc: 0.2747
Epoch 3/100
2022/2022 [==============================] - 3s 1ms/step - loss: 1.6633 - acc: 0.2883 - val_loss: 1.6172 - val_acc: 0.2747
Epoch 4/100
2022/2022 [==============================] - 3s 1ms/step - loss: 1.6442 - acc: 0.3373 - val_loss: 1.6590 - val_acc: 0.2708
Epoch 5/100
2022/2022 [==============================] - 3s 1ms/step - loss: 1.6276 - acc: 0.3037 - val_loss: 1.6717 - val_acc: 0.2708
Epoch 6/100
2022/2022 [==============================] - 3s 1ms/step - loss: 1.6236 - acc: 0.3145 - val_loss: 1.7764 - val_acc: 0.2708
Epoch 7/100
2022/2022 [==============================] - 3s 1ms/step - loss: 1.5910 - acc: 0.3600 - val_loss: 1.2240 - val_acc: 0.5929
Epoch 8/

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 98, 98, 64)        640       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 96, 96, 64)        36928     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 46, 46, 64)        36928     
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 44, 44, 64)        36928     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 22, 22, 64)        0         
__________

In [15]:
test = pd.read_csv('test.csv')

test_image = []

for i in tqdm(range(test.shape[0])):

    img = image.load_img('test_images/'+test['ID'][i], target_size=(100,100,1), grayscale=True)

    img = image.img_to_array(img)

    img = img/255

    test_image.append(img)

test = np.array(test_image)

prediction = model.predict_classes(test)



  0%|          | 0/10142 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/utils.py:104: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
100%|██████████| 10142/10142 [00:31<00:00, 319.25it/s]


In [0]:
download = drive.CreateFile({'id': '1xZTC94m2h-1a1TQAAw1yfwSSh3LWCPgR'})

download.GetContentFile('sample.csv')

# creating submission file

sample = pd.read_csv('sample.csv') #已經有ID的csv檔

sample['label'] = prediction

sample.to_csv('predlabel.csv', header=True, index=False) #加入預測的類別後輸出添加Label的.csv檔

